In [1]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers.merge import Add
from keras.layers.normalization import BatchNormalization
from keras.initializers import he_normal
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
X_train = np.load('../data/X_train.npy', mmap_mode = 'r')
y_train = np.load('../data/y_train_k.npy', mmap_mode = 'r')
X_val = np.load('../data/X_val.npy', mmap_mode = 'r')
y_val = np.load('../data/y_val_k.npy', mmap_mode = 'r')
mean = np.mean(X_train, axis = 0)
std = np.std(X_train, axis = 0)
std[std == 0] = 1
X_train = X_train - mean / std
X_val = X_val - mean / std

In [3]:
def relu_drop_bn_block(ins, neurons, p=.2):
    x = Dense(neurons, kernel_initializer = he_normal(seed=42))(ins)
    x = Activation('relu')(x)
    x = Dropout(p)(x)
    x = BatchNormalization()(x)
    y = Dense(neurons, kernel_initializer = he_normal(seed=42))(x)
    y = Activation('relu')(y)
    y = Dropout(p)(x)
    y = BatchNormalization()(y)
    y = Dense(neurons, kernel_initializer = he_normal(seed=42))(y)
    y = Dropout(p)(y)
    z = Add()([x,y])
    return z

In [4]:
ins = Input(shape = (X_train.shape[1],))
x0 = relu_drop_bn_block(ins, 2048)
x1 = relu_drop_bn_block(x0, 2048)
x2 = relu_drop_bn_block(x1, 1024)
x3 = relu_drop_bn_block(x2, 1024)
x4 = relu_drop_bn_block(x3, 512)
x5 = relu_drop_bn_block(x4, 512)
x6 = relu_drop_bn_block(x5, 256)
x7 = relu_drop_bn_block(x6, 256)
x8 = relu_drop_bn_block(x7, 128)
x9 = relu_drop_bn_block(x8, 128)
outs = Dense(y_train.shape[1], activation = 'softmax', kernel_initializer = he_normal(seed=42))(x9)

model = Model(ins, outs)

In [5]:
opt = Adam()
model.compile(opt, 'categorical_crossentropy', metrics = ['acc'])
stopper = EarlyStopping(monitor='val_acc', patience = 3, mode='auto')
checker = ModelCheckpoint('../src/ensemble/resModels/res_test_best_4', monitor='val_acc', mode='auto', save_best_only = True)

In [6]:
model.fit(X_train, y_train,
          epochs=100, batch_size=32,
          validation_data=(X_val, y_val),
          callbacks = [stopper, checker])

Train on 3428511 samples, validate on 736105 samples
Epoch 1/100
3428511/3428511 [==============================] - 3831s - loss: 0.0415 - acc: 0.9912 - val_loss: 0.1043 - val_acc: 0.9887
Epoch 2/100
3428511/3428511 [==============================] - 3858s - loss: 0.0211 - acc: 0.9955 - val_loss: 0.0935 - val_acc: 0.9919
Epoch 3/100
3428511/3428511 [==============================] - 3819s - loss: 0.0165 - acc: 0.9968 - val_loss: 0.1938 - val_acc: 0.9861
Epoch 4/100
3428511/3428511 [==============================] - 3749s - loss: 0.0139 - acc: 0.9974 - val_loss: 0.1387 - val_acc: 0.9901
Epoch 5/100
3428511/3428511 [==============================] - 3738s - loss: 0.0120 - acc: 0.9978 - val_loss: 0.1073 - val_acc: 0.9914
Epoch 6/100
3428511/3428511 [==============================] - 3735s - loss: 0.0110 - acc: 0.9980 - val_loss: 0.1062 - val_acc: 0.9918


In [3]:
model = load_model('../src/ensemble/res_best.h5')

In [4]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 122)           0                                            
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2048)          251904      input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 2048)          0           dense_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 2048)          0           activation_1[0][0]               
___________________________________________________________________________________________

In [5]:
from sklearn.metrics import accuracy_score

In [6]:
val_preds = model.predict(X_val)
int_preds_val = np.array([np.argmax(r) for r in val_preds])
int_y_val = np.array([np.argmax(r) for r in y_val])
val_acc = accuracy_score(int_y_val, int_preds_val)
print(val_acc)

0.991949518071


In [7]:
train_preds = model.predict(X_train)
int_preds_train = np.array([np.argmax(r) for r in train_preds])
int_y_train = np.array([np.argmax(r) for r in y_train])
train_acc = accuracy_score(int_y_train, int_preds_train)
print(train_acc)

0.991945191367


In [8]:
X_test = np.load('../data/X_test.npy', mmap_mode='r')
y_test = np.load('../data/y_test_k.npy', mmap_mode='r')

In [9]:
test_preds = model.predict(X_test)
int_preds = np.array([np.argmax(r) for r in test_preds])
int_y_test = np.array([np.argmax(r) for r in y_test])
test_acc = accuracy_score(int_y_test, int_preds)
print(test_acc)

0.993193107255


In [11]:
np.save('../data/preds/res_val_preds', val_preds)
np.save('../data/preds/res_train_preds', train_preds)
np.save('../data/preds/res_test_preds', test_preds)